# 共同研究者を管理する

Gakunin RDMプロジェクトの共同研究者の追加や削除、権限の管理を行うタスクです。

## 共同研究者を表示する
Gakunin RDMに登録されている共同研究者を表示します。<br><br>

プロジェクトに登録されているメンバーを確認し、必要に応じてメンバーの編集を行ってください。<br>

共同研究者の編集はGakunin RDMのプロジェクトのメニューにあるメンバーから行ってください。<br>
※【注意】メンバーの権限が「読込み/書込み」「管理者」のいずれかに設定されているメンバーでないとリサーチフローは操作できません。

In [ ]:
# 共同編集者を表示する
import os
import traceback

import panel as pn
import pandas as pd
from IPython.display import display, clear_output
from IPython.core.display import Javascript
from requests.exceptions import RequestException

from library.task_director import TaskDirector
from library.utils.widgets import MessageBox
from library.utils.config import message as msg_config
from library.utils.error import (UnauthorizedError, ProjectNotExist,
                                 UnusableVault, PermissionError)
from library.utils.storage_provider import grdm
from library.utils.html.button import create_button
from library.utils.input import get_grdm_connection_parameters

script_file_name = "manage_collaborators"
notebook_name = script_file_name+'.ipynb'


class CollaboratorManager(TaskDirector):
    """共同研究者の情報を表示するクラスです。

    Attributes:
        instance:
            _form_box (pn.WidgetBox) : フォームを格納する。
            _msg_output (MessageBox) : ユーザーに提示するメッセージを格納する。
            log(traceback.format_exc) :エラーログを取得する。
            doing_task(Callable) :タスク開始によるサブフローステータス管理JSONの更新。
            token(str):トークンID
            api_url(str):APIのURL
            project_id(str):プロジェクトID
            remote_path(str):リモート先のパス
            display_table(Callable):共同研究者の持つ情報を出力する
    """

    def __init__(self, working_path:str) -> None:
        """CollaboratorManager コンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。
        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except PermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        """取得したデータから共同研究者を表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        try:
            self.token, self.project_id = self.get_grdm_params()
            clear_output()
            if not self._msg_output.has_message():
                contents = grdm.get_collaborator_list(grdm.BASE_URL, self.token, self.project_id)
                table = self.display_table(contents)
                button = self.generate_button(self.project_id)

                # フォーム定義
                self._form_box.clear()
                title = msg_config.get('manage_collaborators', 'table_title')
                self._form_box.append(f'# {title}')
                self._form_box.append(table)
                self._form_box.append(button)
                # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
                self.done_task()

        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except ProjectNotExist:
            message = msg_config.get('form', 'project_id_not_exist').format(self.project_id)
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension('tabulator')
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def generate_button(self, project_id:str) -> pn.pane.HTML:
        """メンバー一覧のページを表示するボタンを作成するメソッドです。

        Args:
            project_id(str):プロジェクトID

        Returns:
            pn.pane.HTML:ボタンを表示させる。
        """
        button_width = 500
        obj = create_button(
            url=grdm.build_collaborator_url(grdm.BASE_URL, project_id),
            msg=msg_config.get('manage_collaborators', 'go_grdm'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        return pn.pane.HTML(obj, width=button_width)

    def display_table(self, contents:dict) -> pn.widgets.Tabulator:
        """共同研究者の持つ情報を出力するメソッドです。

        共同研究者の名前や管理者、書き込み、読み取りの内、どの権限を持っているかなどを出力する。

        Args:
            contents(dict):共同管理者の情報

        Returns:
            pn.widgets.Tabulator:共同研究者の情報を表示する表
        """
        display_permission = {
                "admin": msg_config.get('manage_collaborators', 'admin'),
                "write": msg_config.get('manage_collaborators', 'write'),
                "read": msg_config.get('manage_collaborators', 'read')
            }

        name_list = []
        permission_list = []
        for full_name, permission in contents.items():
            name_list.append(full_name)
            permission_list.append(display_permission[permission])

        df = pd.DataFrame({
            msg_config.get('manage_collaborators', 'name_title'): name_list,
            msg_config.get('manage_collaborators', 'permission_title'): permission_list
        })
        return pn.widgets.Tabulator(
            df,
            show_index=False,
            header_align='center',
            text_align='center',
            configuration={'columnDefaults': {'headerSort': False}},
            editors={
                msg_config.get('manage_collaborators', 'name_title'): None,
                msg_config.get('manage_collaborators', 'permission_title'): None
            }
        )

CollaboratorManager(working_path=os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

script_file_name = "manage_collaborators"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()